# Importing Packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from arch import arch_model #GARCH Models
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import numpy as np
import math
import scipy.stats
from statistics import NormalDist
from scipy.stats import t
from scipy.stats import f
from datetime import datetime
from scipy.stats import shapiro #Shapiro Test
from scipy import stats #t-test
import statsmodels.api as sm #Ljung-Box Test
from scipy.stats import chi2

# Price and Returns

In [2]:
start_date = '2006-01-01' #yyyy-mm-dd
end_date = '2021-01-01'


#Philippines
PH = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/2006-2021/PSEi.csv')
PH['Date'] = pd.to_datetime(PH['Date'])
PH = PH.rename(columns={'Price': 'Close'})
PH = PH.replace(',','', regex=True)
PH['Close'] = PH['Close'].astype(float, errors = 'raise')
mask = (PH['Date'] >= start_date) & (PH['Date'] <= end_date)
PH = PH.loc[mask]
PH = PH.set_index('Date')
PH = PH.sort_index(axis=0, ascending = False)

In [3]:
def get_returns(df,d):
    '''
    The function obtains the log returns of the asset shifted d days
    
    PARAMETERS
    ----------
    df : pandas.DataFrame
        The data frame contains data of a chosen stock index 
        (Stock index must be arrange in DESCENDING ORDER by DATE)
    d : int
        The dth day being forecast
        (Assumed to be 1 for most cases)
        
    RETURNS
    -------
    df : pandas.DataFrame
        The data frame returns an updated data frame containing the
        'Returns' column
    '''
    df['Previous'] = df['Close'].shift(-d)
    df['Returns'] = np.log(df['Close']/df['Previous'])*100
    display(df)
    return df

PH = get_returns(PH, 1)

,Open,High,Low,Close,Unnamed: 5,Unnamed: 6,Previous,Returns
Date,,,,,,,,
2020-12-29,7123.12,7173.27,7120.82,7139.71,NaN,NaN,7122.25,0.244847
2020-12-28,7142.95,7142.95,7058.66,7122.25,NaN,NaN,7204.38,-1.146549
2020-12-23,7169.05,7206.46,7075.18,7204.38,NaN,NaN,7202.39,0.027626
2020-12-22,7206.33,7206.54,7081.14,7202.39,NaN,NaN,7224.89,-0.311909
2020-12-21,7244.85,7251.02,7157.88,7224.89,NaN,NaN,7272.80,-0.660935
...,...,...,...,...,...,...,...,...
2006-01-06,2150.57,2157.18,2132.05,2133.79,NaN,NaN,2150.93,-0.800057
2006-01-05,2138.29,2150.93,2129.34,2150.93,NaN,NaN,2138.05,0.600611
2006-01-04,2129.97,2144.04,2129.97,2138.05,NaN,NaN,2109.79,1.330578


# Diebold-Mariano Test

In [68]:
# Author   : John Tsang
# Date     : December 7th, 2017
# Purpose  : Implement the Diebold-Mariano Test (DM test) to compare 
#            forecast accuracy
# Input    : 1) actual_lst: the list of actual values
#            2) pred1_lst : the first list of predicted values
#            3) pred2_lst : the second list of predicted values
#            4) h         : the number of stpes ahead
#            5) crit      : a string specifying the criterion 
#                             i)  MSE : the mean squared error
#                            ii)  MAD : the mean absolute deviation
#                           iii) MAPE : the mean absolute percentage error
#                            iv) poly : use power function to weigh the errors
#            6) poly      : the power for crit power 
#                           (it is only meaningful when crit is "poly")
# Condition: 1) length of actual_lst, pred1_lst and pred2_lst is equal
#            2) h must be an integer and it must be greater than 0 and less than 
#               the length of actual_lst.
#            3) crit must take the 4 values specified in Input
#            4) Each value of actual_lst, pred1_lst and pred2_lst must
#               be numerical values. Missing values will not be accepted.
#            5) power must be a numerical value.
# Return   : a named-tuple of 2 elements
#            1) p_value : the p-value of the DM test
#            2) DM      : the test statistics of the DM test
##########################################################
# References:
#
# Harvey, D., Leybourne, S., & Newbold, P. (1997). Testing the equality of 
#   prediction mean squared errors. International Journal of forecasting, 
#   13(2), 281-291.
#
# Diebold, F. X. and Mariano, R. S. (1995), Comparing predictive accuracy, 
#   Journal of business & economic statistics 13(3), 253-264.
#
##########################################################
def dm_test(actual_lst, pred1_lst, pred2_lst, h = 1, crit="MSE", power = 2):
    # Routine for checking errors
    def error_check():
        rt = 0
        msg = ""
        # Check if h is an integer
        if (not isinstance(h, int)):
            rt = -1
            msg = "The type of the number of steps ahead (h) is not an integer."
            return (rt,msg)
        # Check the range of h
        if (h < 1):
            rt = -1
            msg = "The number of steps ahead (h) is not large enough."
            return (rt,msg)
        len_act = len(actual_lst)
        len_p1  = len(pred1_lst)
        len_p2  = len(pred2_lst)
        # Check if lengths of actual values and predicted values are equal
        if (len_act != len_p1 or len_p1 != len_p2 or len_act != len_p2):
            rt = -1
            msg = "Lengths of actual_lst, pred1_lst and pred2_lst do not match."
            return (rt,msg)
        # Check range of h
        if (h >= len_act):
            rt = -1
            msg = "The number of steps ahead is too large."
            return (rt,msg)
        # Check if criterion supported
        if (crit != "MSE" and crit != "MAPE" and crit != "MAD" and crit != "poly"):
            rt = -1
            msg = "The criterion is not supported."
            return (rt,msg)  
        # Check if every value of the input lists are numerical values
        from re import compile as re_compile
        comp = re_compile("^\d+?\.\d+?$")  
        def compiled_regex(s):
            """ Returns True is string is a number. """
            if comp.match(s) is None:
                return s.isdigit()
            return True
        for actual, pred1, pred2 in zip(actual_lst, pred1_lst, pred2_lst):
            is_actual_ok = compiled_regex(str(abs(actual)))
            is_pred1_ok = compiled_regex(str(abs(pred1)))
            is_pred2_ok = compiled_regex(str(abs(pred2)))
            if (not (is_actual_ok and is_pred1_ok and is_pred2_ok)):  
                msg = "An element in the actual_lst, pred1_lst or pred2_lst is not numeric."
                rt = -1
                return (rt,msg)
        return (rt,msg)
    
    # Error check
    error_code = error_check()
    # Raise error if cannot pass error check
    if (error_code[0] == -1):
        raise SyntaxError(error_code[1])
        return
    # Import libraries
    from scipy.stats import t
    import collections
    import pandas as pd
    import numpy as np
    
    # Initialise lists
    e1_lst = []
    e2_lst = []
    d_lst  = []
    
    # convert every value of the lists into real values
    actual_lst = pd.Series(actual_lst).apply(lambda x: float(x)).tolist()
    pred1_lst = pd.Series(pred1_lst).apply(lambda x: float(x)).tolist()
    pred2_lst = pd.Series(pred2_lst).apply(lambda x: float(x)).tolist()
    
    # Length of lists (as real numbers)
    T = float(len(actual_lst))
    
    # construct d according to crit
    if (crit == "MSE"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append((actual - p1)**2)
            e2_lst.append((actual - p2)**2)
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "MAD"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(abs(actual - p1))
            e2_lst.append(abs(actual - p2))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "MAPE"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(abs((actual - p1)/actual))
            e2_lst.append(abs((actual - p2)/actual))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "poly"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(((actual - p1))**(power))
            e2_lst.append(((actual - p2))**(power))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)    
    
    # Mean of d        
    mean_d = pd.Series(d_lst).mean()
    
    # Find autocovariance and construct DM test statistics
    def autocovariance(Xi, N, k, Xs):
        autoCov = 0
        T = float(N)
        for i in np.arange(0, N-k):
              autoCov += ((Xi[i+k])-Xs)*(Xi[i]-Xs)
        return (1/(T))*autoCov
    gamma = []
    for lag in range(0,h):
        gamma.append(autocovariance(d_lst,len(d_lst),lag,mean_d)) # 0, 1, 2
    V_d = (gamma[0] + 2*sum(gamma[1:]))/T
    DM_stat=V_d**(-0.5)*mean_d
    harvey_adj=((T+1-2*h+h*(h-1)/T)/T)**(0.5)
    DM_stat = harvey_adj*DM_stat
    # Find p-value
    p_value = 2*t.cdf(-abs(DM_stat), df = T - 1)
    
    # Construct named tuple for return
    dm_return = collections.namedtuple('dm_return', 'DM p_value')
    
    rt = dm_return(DM = DM_stat, p_value = p_value)
    
    return rt

## Custom DM-Test Function

In [48]:
# def DM_test(df, model_1, model_2, test_size):
    # returns = np.array(df['Returns'])[:test_size]
    # arr_model_1 = model_1[['Forecasted VaR','Volatility']].to_numpy()
    # arr_model_2 = model_2[['Forecasted VaR','Volatility']].to_numpy()
    # dm = dm_test(returns, arr_model_1, arr_model_2, h = 1, crit = 'MSE')
    # return dm

In [86]:
# def DM_test(df, model_1, model_2, test_size):
#     returns = np.array(df['Returns'])[:test_size]
    
#     arr_model_1_dropped = model_1.drop(['Date'], axis = 1)
#     arr_model_2_dropped = model_1.drop(['Date'], axis = 1)
    
#     arr_model_1 = arr_model_1_dropped[arr_model_1_dropped.columns[0]].to_numpy()
#     arr_model_2 = arr_model_2_dropped[arr_model_2_dropped.columns[0]].to_numpy()
    
#     dm = dm_test(returns, arr_model_1, arr_model_2, h = 1, crit = 'MSE')
    
#     return dm

In [99]:
# arr_model_1_dropped = PH_fw_01_dn.drop(['Date'], axis = 1)
# print(arr_model_1_dropped)

# arr_model_1_zero = arr_model_1_dropped.iloc[:, 0].to_numpy()
# print(arr_model_1_zero)

# arr_model_2_dropped = PH_fw_01_hs.drop(['Date'], axis = 1)
# print(arr_model_2_dropped)

# arr_model_2_zero = arr_model_2_dropped.iloc[:,0].to_numpy()
# print(arr_model_2_zero)

# returns = np.array(PH['Returns'])[:781]
# print(returns)

# dm = dm_test(returns, arr_model_1_zero, arr_model_2_zero, h = 1, crit = 'MSE')

In [96]:
def DM_test(country, model_1, model_2, t):
    
    arr_model_1_dropped = model_1.drop(['Date'], axis = 1)
    arr_model_1_zero = arr_model_1_dropped.iloc[:, 0].to_numpy()

    arr_model_2_dropped = model_2.drop(['Date'], axis = 1)
    arr_model_2_zero = arr_model_2_dropped.iloc[:,0].to_numpy()

    returns = np.array(PH['Returns'])[:t]

    dm = dm_test(returns, arr_model_1_zero, arr_model_2_zero, h = 1, crit = 'MSE')
    
    return dm

# Fixed Window

## 0.01%

In [33]:
PH_fw_01_dn = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/DN/0.01%25/PH_VaR_FW-DN-0.01%25.csv')
PH_fw_01_hs = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/HS/0.01%25/PH_VaR_FW-HS-0.01%25.csv')
PH_fw_01_brw = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/BRW/0.01%25/PH_VaR_FW-BRW-0.01%25.csv')
PH_fw_01_garch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/GARCH/0.01%25/PH_VaR_FW-G-0.01%25.csv')
PH_fw_01_earch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EARCH/0.01%25/PH_VaR_FW-EA-0.01%25.csv')
PH_fw_01_egarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EGARCH/0.01%25/PH_VaR_FW-E-0.01%25.csv')
PH_fw_01_igarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/IGARCH/IGARCH-0.01%25-FW/PH_fw_FW-I-0.01%25.csv')
PH_fw_01_figarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH/FIGARCH-0.01%25-FW/PH_fw_FW-FI-0.01%25.csv')
PH_fw_01_figarch0633 = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH0.633/FIGARCH0.633-0.01%25-FW/PH_fw_FW-FI0.633-0.01%25.csv')

In [105]:
# DN
print(DM_test(PH, PH_fw_01_dn, PH_fw_01_hs, 781))
print(DM_test(PH, PH_fw_01_dn, PH_fw_01_brw, 781))
print(DM_test(PH, PH_fw_01_dn, PH_fw_01_garch, 781))
print(DM_test(PH, PH_fw_01_dn, PH_fw_01_earch, 781))
print(DM_test(PH, PH_fw_01_dn, PH_fw_01_egarch, 781))
print(DM_test(PH, PH_fw_01_dn, PH_fw_01_igarch, 781))
print(DM_test(PH, PH_fw_01_dn, PH_fw_01_figarch, 781))
print(DM_test(PH, PH_fw_01_dn, PH_fw_01_figarch0633, 781))

dm_return(DM=-84.76094441278836, p_value=0.0)
dm_return(DM=-15.053286352980448, p_value=3.802884654712075e-45)
dm_return(DM=-3.5160764065391885, p_value=0.000463269504543097)
dm_return(DM=-0.3407536255984439, p_value=0.7333808226343581)
dm_return(DM=-3.0027389360096537, p_value=0.002761079426311385)
dm_return(DM=-2.5225420090988746, p_value=0.011848865458636981)
dm_return(DM=-4.119268454373675, p_value=4.206856942093485e-05)
dm_return(DM=-3.478442168567482, p_value=0.0005322941057406609)


In [106]:
# HS
print(DM_test(PH, PH_fw_01_hs, PH_fw_01_brw, 781))
print(DM_test(PH, PH_fw_01_hs, PH_fw_01_garch, 781))
print(DM_test(PH, PH_fw_01_hs, PH_fw_01_earch, 781))
print(DM_test(PH, PH_fw_01_hs, PH_fw_01_egarch, 781))
print(DM_test(PH, PH_fw_01_hs, PH_fw_01_igarch, 781))
print(DM_test(PH, PH_fw_01_hs, PH_fw_01_figarch, 781))
print(DM_test(PH, PH_fw_01_hs, PH_fw_01_figarch0633, 781))

dm_return(DM=18.94220059244299, p_value=4.0429627834884475e-66)
dm_return(DM=8.809822591543611, p_value=8.040640588118636e-18)
dm_return(DM=42.17542555544652, p_value=2.0864308095889964e-203)
dm_return(DM=15.527098773815593, p_value=1.4099730855283968e-47)
dm_return(DM=3.7545390157290184, p_value=0.00018656712799172535)
dm_return(DM=8.529485121656059, p_value=7.594157434321829e-17)
dm_return(DM=5.588212507324295, p_value=3.171532193637059e-08)


In [108]:
# BRW
print(DM_test(PH, PH_fw_01_brw, PH_fw_01_garch, 781))
print(DM_test(PH, PH_fw_01_brw, PH_fw_01_earch, 781))
print(DM_test(PH, PH_fw_01_brw, PH_fw_01_egarch, 781))
print(DM_test(PH, PH_fw_01_brw, PH_fw_01_igarch, 781))
print(DM_test(PH, PH_fw_01_brw, PH_fw_01_figarch, 781))
print(DM_test(PH, PH_fw_01_brw, PH_fw_01_figarch0633, 781))

dm_return(DM=1.5995394538399035, p_value=0.11010573741246787)
dm_return(DM=11.776154357683952, p_value=1.4011950808339368e-29)
dm_return(DM=4.446756791665255, p_value=9.982971774672081e-06)
dm_return(DM=0.10259653836027594, p_value=0.9183095603092271)
dm_return(DM=1.1561067189980616, p_value=0.24799148918751432)
dm_return(DM=0.31091426096091757, p_value=0.7559488709640833)


In [109]:
# GARCH
print(DM_test(PH, PH_fw_01_garch, PH_fw_01_earch, 781))
print(DM_test(PH, PH_fw_01_garch, PH_fw_01_egarch, 781))
print(DM_test(PH, PH_fw_01_garch, PH_fw_01_igarch, 781))
print(DM_test(PH, PH_fw_01_garch, PH_fw_01_figarch, 781))
print(DM_test(PH, PH_fw_01_garch, PH_fw_01_figarch0633, 781))

dm_return(DM=3.7114946865788014, p_value=0.00022068395056166747)
dm_return(DM=4.312802569416899, p_value=1.8188061973717996e-05)
dm_return(DM=-1.0709251868673513, p_value=0.28453432717850363)
dm_return(DM=-2.902676742107884, p_value=0.0038042055066395174)
dm_return(DM=-2.09678692908013, p_value=0.03633393090568196)


In [110]:
# EARCH
print(DM_test(PH, PH_fw_01_earch, PH_fw_01_egarch, 781))
print(DM_test(PH, PH_fw_01_earch, PH_fw_01_igarch, 781))
print(DM_test(PH, PH_fw_01_earch, PH_fw_01_figarch, 781))
print(DM_test(PH, PH_fw_01_earch, PH_fw_01_figarch0633, 781))

dm_return(DM=-3.1739866393547196, p_value=0.0015626156520959545)
dm_return(DM=-2.6829202689723264, p_value=0.0074527947285733176)
dm_return(DM=-4.416379770545266, p_value=1.1453706535348804e-05)
dm_return(DM=-3.750285498250799, p_value=0.00018970312206328214)


In [111]:
# EGARCH
print(DM_test(PH, PH_fw_01_egarch, PH_fw_01_igarch, 781))
print(DM_test(PH, PH_fw_01_egarch, PH_fw_01_figarch, 781))
print(DM_test(PH, PH_fw_01_egarch, PH_fw_01_figarch0633, 781))

dm_return(DM=-1.9929662268173078, p_value=0.046613140199769046)
dm_return(DM=-5.983446314050501, p_value=3.327758248042429e-09)
dm_return(DM=-3.5262405924382056, p_value=0.0004461168621870455)


In [113]:
# IGARCH
print(DM_test(PH, PH_fw_01_igarch, PH_fw_01_figarch, 781))
print(DM_test(PH, PH_fw_01_igarch, PH_fw_01_figarch0633, 781))

dm_return(DM=0.7507478225646607, p_value=0.453030854345321)
dm_return(DM=0.285415742167007, p_value=0.7754014208364659)


In [114]:
# FIGARCH
print(DM_test(PH, PH_fw_01_figarch, PH_fw_01_figarch0633, 781))

dm_return(DM=-1.4622850431498933, p_value=0.14406589430164743)


## 1.00 %

In [115]:
PH_fw_1_dn = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/DN/1%25/PH_VaR_FW-DN-1%25.csv')
PH_fw_1_hs = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/HS/1%25/PH_VaR_FW-HS-1%25.csv')
PH_fw_1_brw = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/BRW/1%25/PH_VaR_FW-BRW-1%25.csv')
PH_fw_1_garch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/GARCH/1%25/PH_VaR_FW-G-1%25.csv')
PH_fw_1_earch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EARCH/1%25/PH_VaR_FW-EA-1%25.csv')
PH_fw_1_egarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EGARCH/1%25/PH_VaR_FW-E-1%25.csv')
PH_fw_1_igarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/IGARCH/IGARCH-1%25-FW/PH_fw_FW-I-1%25.csv')
PH_fw_1_figarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH/FIGARCH-1%25-FW/PH_fw_FW-FI-1%25.csv')
PH_fw_1_figarch0633 = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH0.633/FIGARCH0.633-1%25-FW/PH_fw_FW-FI0.633-1%25.csv')

In [116]:
# DN
print(DM_test(PH, PH_fw_1_dn, PH_fw_1_hs, 781))
print(DM_test(PH, PH_fw_1_dn, PH_fw_1_brw, 781))
print(DM_test(PH, PH_fw_1_dn, PH_fw_1_garch, 781))
print(DM_test(PH, PH_fw_1_dn, PH_fw_1_earch, 781))
print(DM_test(PH, PH_fw_1_dn, PH_fw_1_egarch, 781))
print(DM_test(PH, PH_fw_1_dn, PH_fw_1_igarch, 781))
print(DM_test(PH, PH_fw_1_dn, PH_fw_1_figarch, 781))
print(DM_test(PH, PH_fw_1_dn, PH_fw_1_figarch0633, 781))

dm_return(DM=-16.54129662994321, p_value=6.576179620194045e-53)
dm_return(DM=-16.94691993532945, p_value=4.371381563956138e-55)
dm_return(DM=-3.4762409419102585, p_value=0.0005366147356787624)
dm_return(DM=-0.3698274620918392, p_value=0.7116114396358482)
dm_return(DM=-2.9524231775836363, p_value=0.0032475299486857015)
dm_return(DM=-2.514414696307132, p_value=0.012123185950582779)
dm_return(DM=-4.072425941464794, p_value=5.127709659843171e-05)
dm_return(DM=-3.4423774298452146, p_value=0.0006073471416170175)


In [117]:
# HS
print(DM_test(PH, PH_fw_1_hs, PH_fw_1_brw, 781))
print(DM_test(PH, PH_fw_1_hs, PH_fw_1_garch, 781))
print(DM_test(PH, PH_fw_1_hs, PH_fw_1_earch, 781))
print(DM_test(PH, PH_fw_1_hs, PH_fw_1_egarch, 781))
print(DM_test(PH, PH_fw_1_hs, PH_fw_1_igarch, 781))
print(DM_test(PH, PH_fw_1_hs, PH_fw_1_figarch, 781))
print(DM_test(PH, PH_fw_1_hs, PH_fw_1_figarch0633, 781))

dm_return(DM=-16.20580064870336, p_value=3.9806422003189624e-51)
dm_return(DM=-3.11505186357847, p_value=0.0019064326075571953)
dm_return(DM=0.9647173174962824, p_value=0.3349854149972894)
dm_return(DM=-2.4126494046985583, p_value=0.016066778729425434)
dm_return(DM=-2.3374589072765852, p_value=0.019667378289817614)
dm_return(DM=-3.7006586785026925, p_value=0.00023015379725394884)
dm_return(DM=-3.181523495490357, p_value=0.0015230334969678699)


In [118]:
# BRW
print(DM_test(PH, PH_fw_1_brw, PH_fw_1_garch, 781))
print(DM_test(PH, PH_fw_1_brw, PH_fw_1_earch, 781))
print(DM_test(PH, PH_fw_1_brw, PH_fw_1_egarch, 781))
print(DM_test(PH, PH_fw_1_brw, PH_fw_1_igarch, 781))
print(DM_test(PH, PH_fw_1_brw, PH_fw_1_figarch, 781))
print(DM_test(PH, PH_fw_1_brw, PH_fw_1_figarch0633, 781))

dm_return(DM=2.543444740050071, p_value=0.011168507452243822)
dm_return(DM=13.541488230017201, p_value=1.124339368275917e-37)
dm_return(DM=5.812352203056531, p_value=8.976521117407938e-09)
dm_return(DM=0.5341017588309703, p_value=0.593423316814017)
dm_return(DM=2.102366849678673, p_value=0.035841133601464334)
dm_return(DM=0.9599403274462078, p_value=0.33738271165608935)


In [119]:
# GARCH
print(DM_test(PH, PH_fw_1_garch, PH_fw_1_earch, 781))
print(DM_test(PH, PH_fw_1_garch, PH_fw_1_egarch, 781))
print(DM_test(PH, PH_fw_1_garch, PH_fw_1_igarch, 781))
print(DM_test(PH, PH_fw_1_garch, PH_fw_1_figarch, 781))
print(DM_test(PH, PH_fw_1_garch, PH_fw_1_figarch0633, 781))

dm_return(DM=3.665761628518185, p_value=0.0002633134997781572)
dm_return(DM=4.29769954756112, p_value=1.944133842780066e-05)
dm_return(DM=-1.117138702936723, p_value=0.2642790109892734)
dm_return(DM=-2.995084867829833, p_value=0.002830498233480178)
dm_return(DM=-2.1351544823619832, p_value=0.033059366494270555)


In [120]:
# EARCH
print(DM_test(PH, PH_fw_1_earch, PH_fw_1_egarch, 781))
print(DM_test(PH, PH_fw_1_earch, PH_fw_1_igarch, 781))
print(DM_test(PH, PH_fw_1_earch, PH_fw_1_figarch, 781))
print(DM_test(PH, PH_fw_1_earch, PH_fw_1_figarch0633, 781))

dm_return(DM=-3.1116957699442946, p_value=0.001927965218243067)
dm_return(DM=-2.669725802867528, p_value=0.007749252771569769)
dm_return(DM=-4.360345023856617, p_value=1.4726963119847188e-05)
dm_return(DM=-3.7050299634527017, p_value=0.00022628854606168217)


In [121]:
# EGARCH
print(DM_test(PH, PH_fw_1_egarch, PH_fw_1_igarch, 781))
print(DM_test(PH, PH_fw_1_egarch, PH_fw_1_figarch, 781))
print(DM_test(PH, PH_fw_1_egarch, PH_fw_1_figarch0633, 781))

dm_return(DM=-2.015330095063527, p_value=0.04421263933645943)
dm_return(DM=-5.9519701331689765, p_value=4.001705410621301e-09)
dm_return(DM=-3.5164531444732257, p_value=0.00046262288616328424)


In [122]:
# IGARCH
print(DM_test(PH, PH_fw_1_igarch, PH_fw_1_figarch, 781))
print(DM_test(PH, PH_fw_1_igarch, PH_fw_1_figarch0633, 781))

dm_return(DM=0.7905569702932714, p_value=0.42944285625005163)
dm_return(DM=0.33157117185800106, p_value=0.7403021568497206)


In [123]:
# FIGARCH
print(DM_test(PH, PH_fw_1_figarch, PH_fw_1_figarch0633, 781))

dm_return(DM=-1.485286904961992, p_value=0.13787186692250467)
